
 **PREDICCION SUPERVIVENCIA TITANIC** 



In [1]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import keras
from keras.models import Sequential
from keras.layers import Dense
from sklearn.metrics import accuracy_score
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold




## Importando data ##


In [2]:
url_test= 'https://raw.githubusercontent.com/Daniel11182092/LAB1_StatisticalL2/main/LAB1_StatisticalL2/test.csv'
url_train= 'https://raw.githubusercontent.com/Daniel11182092/LAB1_StatisticalL2/main/LAB1_StatisticalL2/train.csv'

In [3]:
df_test_original = pd.read_csv(url_test)
df_train_original= pd.read_csv(url_train)

df_test= df_test_original 
df_train= df_train_original




## Analisis de la data ##

In [4]:
df_test.head()


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [5]:
df_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [6]:
print('Validando cantidad de datos')
print(df_train.shape)
print(df_test.shape)

Validando cantidad de datos
(891, 12)
(418, 11)


In [7]:
print('Validando Tipos de datos')
print(df_train.info())
print(df_test.info())

Validando Tipos de datos
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non

In [8]:
print('Validando datos faltantes')
print(pd.isnull(df_train).sum())
print(pd.isnull(df_test).sum())

Validando datos faltantes
PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64
PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64


In [9]:
print('Validando estadisticas de los dataset')
print(df_train.describe())
print(df_test.describe())

Validando estadisticas de los dataset
       PassengerId    Survived      Pclass         Age       SibSp  \
count   891.000000  891.000000  891.000000  714.000000  891.000000   
mean    446.000000    0.383838    2.308642   29.699118    0.523008   
std     257.353842    0.486592    0.836071   14.526497    1.102743   
min       1.000000    0.000000    1.000000    0.420000    0.000000   
25%     223.500000    0.000000    2.000000   20.125000    0.000000   
50%     446.000000    0.000000    3.000000   28.000000    0.000000   
75%     668.500000    1.000000    3.000000   38.000000    1.000000   
max     891.000000    1.000000    3.000000   80.000000    8.000000   

            Parch        Fare  
count  891.000000  891.000000  
mean     0.381594   32.204208  
std      0.806057   49.693429  
min      0.000000    0.000000  
25%      0.000000    7.910400  
50%      0.000000   14.454200  
75%      0.000000   31.000000  
max      6.000000  512.329200  
       PassengerId      Pclass         Age 

## Preprocesamiento de datos ##





### *Cambio de datos para columnas Sexo y Embarque

In [10]:
df_train['Sex'].replace(['female','male'],[0,1], inplace= True)
df_test ['Sex'].replace(['female','male'],[0,1], inplace= True)

In [11]:
df_train['Embarked'].replace(['Q','S', 'C'],[0,1,2], inplace= True)
df_test ['Embarked'].replace(['Q','S', 'C'],[0,1,2], inplace= True)

In [ ]:
df_train

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",1,22.0,1,0,A/5 21171,7.2500,NaN,1.0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,38.0,1,0,PC 17599,71.2833,C85,2.0
2,3,1,3,"Heikkinen, Miss. Laina",0,26.0,0,0,STON/O2. 3101282,7.9250,NaN,1.0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,35.0,1,0,113803,53.1000,C123,1.0
4,5,0,3,"Allen, Mr. William Henry",1,35.0,0,0,373450,8.0500,NaN,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",1,27.0,0,0,211536,13.0000,NaN,1.0
887,888,1,1,"Graham, Miss. Margaret Edith",0,19.0,0,0,112053,30.0000,B42,1.0
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",0,NaN,1,2,W./C. 6607,23.4500,NaN,1.0
889,890,1,1,"Behr, Mr. Karl Howell",1,26.0,0,0,111369,30.0000,C148,2.0


In [12]:
df_test

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",1,34.5,0,0,330911,7.8292,NaN,0
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",0,47.0,1,0,363272,7.0000,NaN,1
2,894,2,"Myles, Mr. Thomas Francis",1,62.0,0,0,240276,9.6875,NaN,0
3,895,3,"Wirz, Mr. Albert",1,27.0,0,0,315154,8.6625,NaN,1
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",0,22.0,1,1,3101298,12.2875,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",1,NaN,0,0,A.5. 3236,8.0500,NaN,1
414,1306,1,"Oliva y Ocana, Dona. Fermina",0,39.0,0,0,PC 17758,108.9000,C105,2
415,1307,3,"Saether, Mr. Simon Sivertsen",1,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,1
416,1308,3,"Ware, Mr. Frederick",1,NaN,0,0,359309,8.0500,NaN,1


## *Remplazando datos faltantes por imputacion de media en columna fare df_test

In [13]:
print(df_test['Fare'].mean())
promedio = 33.5
df_test['Fare'] = df_train['Fare'].replace(np.nan, promedio)



35.627188489208635


## *Remplazando datos faltantes por imputacion de media en columna edad

In [14]:
print(df_train['Age'].mean())
print(df_test['Age'].mean())
promedio = 30
df_train['Age'] = df_train['Age'].replace(np.nan, promedio)
df_test['Age'] = df_train['Age'].replace(np.nan, promedio)

29.69911764705882
30.272590361445783


## *Eliminando Columna Cabin considerando la cantidad de datos perdidos

In [15]:
df_train.drop(['Cabin'],axis=1 , inplace=True)
df_test.drop(['Cabin'],axis=1 , inplace=True)

## *Eliminando Columnas que no son necesarias para analisis 

In [16]:
df_train = df_train.drop(['PassengerId', 'Name','Ticket'], axis=1)
df_test = df_test.drop(['Name', 'Ticket'], axis=1)


# *Eliminando filas con datos perdidos en df_train

In [17]:
df_train.dropna(axis=0, how='any', inplace=True)

## Verificando que los datos esten limpios 

In [18]:
print('Validando datos faltantes')
print(pd.isnull(df_train).sum())
print(pd.isnull(df_test).sum())

Validando datos faltantes
Survived    0
Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Embarked    0
dtype: int64
PassengerId    0
Pclass         0
Sex            0
Age            0
SibSp          0
Parch          0
Fare           0
Embarked       0
dtype: int64


In [19]:
df_train

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,1,22.0,1,0,7.2500,1.0
1,1,1,0,38.0,1,0,71.2833,2.0
2,1,3,0,26.0,0,0,7.9250,1.0
3,1,1,0,35.0,1,0,53.1000,1.0
4,0,3,1,35.0,0,0,8.0500,1.0
...,...,...,...,...,...,...,...,...
886,0,2,1,27.0,0,0,13.0000,1.0
887,1,1,0,19.0,0,0,30.0000,1.0
888,0,3,0,30.0,1,2,23.4500,1.0
889,1,1,1,26.0,0,0,30.0000,2.0


In [20]:
df_test

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,892,3,1,22.0,0,0,7.2500,0
1,893,3,0,38.0,1,0,71.2833,1
2,894,2,1,26.0,0,0,7.9250,0
3,895,3,1,35.0,0,0,53.1000,1
4,896,3,0,35.0,1,1,8.0500,1
...,...,...,...,...,...,...,...,...
413,1305,3,1,30.0,0,0,0.0000,1
414,1306,1,0,44.0,0,0,7.9250,2
415,1307,3,1,30.0,0,0,8.0500,1
416,1308,3,1,34.0,0,0,32.5000,1


## **ARQUITECTURAS DE REDES**

In [21]:
X_train = df_train.drop("Survived", axis=1)
y_train = df_train["Survived"]
X_test  = df_test.drop("PassengerId", axis=1).copy()
X_train.shape, y_train.shape, X_test.shape




((889, 7), (889,), (418, 7))

In [23]:
X_train.shape[1]
inputs = X_train.shape[1]


### 1. Arquitectura 1

In [24]:
#base secuencial para definir la estructura de la Red 
clasificador = Sequential()

#primera capa oclta
clasificador.add(Dense(input_dim = inputs, units=6, activation='relu', kernel_initializer='uniform'))

#segunda capa oculta
clasificador.add(Dense(units=6, activation='relu', kernel_initializer='uniform'))

#capa de salida
clasificador.add(Dense(units=1, activation='sigmoid', kernel_initializer='uniform'))

#parametros de optimización
clasificador.compile(optimizer='SGD', loss='binary_crossentropy', metrics=['accuracy'])

#parametros de entrenamiento.
clasificador.fit(X_train, y_train, batch_size=25, epochs=500)

Epoch 1/500
36/36 [==============================] - 1s 4ms/step - loss: 0.6911 - accuracy: 0.6029
Epoch 2/500
36/36 [==============================] - 0s 3ms/step - loss: 0.6869 - accuracy: 0.6175
Epoch 3/500
36/36 [==============================] - 0s 3ms/step - loss: 0.6835 - accuracy: 0.6175
Epoch 4/500
36/36 [==============================] - 0s 3ms/step - loss: 0.6805 - accuracy: 0.6175
Epoch 5/500
36/36 [==============================] - 0s 2ms/step - loss: 0.6781 - accuracy: 0.6175
Epoch 6/500
36/36 [==============================] - 0s 2ms/step - loss: 0.6760 - accuracy: 0.6175
Epoch 7/500
36/36 [==============================] - 0s 3ms/step - loss: 0.6742 - accuracy: 0.6175
Epoch 8/500
36/36 [==============================] - 0s 5ms/step - loss: 0.6727 - accuracy: 0.6175
Epoch 9/500
36/36 [==============================] - 0s 6ms/step - loss: 0.6716 - accuracy: 0.6175
Epoch 10/500
36/36 [==============================] - 0s 5ms/step - loss: 0.6706 - accuracy: 0.6175
Epoch 11/

In [75]:
# Guardardamos la red.
path = 'C:/Users/Daniel Soto/Desktop/MAESTRIA BI/2022/Statistical Learning/LAB1_StatisticalL2' 
clasificador.save(path+'arq1.h5')

# Cargamos la red.
new_model1 = keras.models.load_model(path+'arq1.h5')







In [76]:



#predicciones
y_preds1 = new_model1.predict(X_test)
y_preds1

array([[0.11438063],
       [0.5226573 ],
       [0.09026831],
       [0.2366986 ],
       [0.10089067],
       [0.06721699],
       [0.5143546 ],
       [0.48826316],
       [0.39291796],
       [0.16070092],
       [0.55157936],
       [0.08267808],
       [0.51499987],
       [0.06415182],
       [0.60463387],
       [0.16630119],
       [0.4344325 ],
       [0.06987524],
       [0.22956747],
       [0.29374856],
       [0.08527464],
       [0.05125186],
       [0.74795413],
       [0.22473237],
       [0.705017  ],
       [0.04642525],
       [0.43447256],
       [0.71022624],
       [0.07355341],
       [0.06164682],
       [0.05325791],
       [0.57389295],
       [0.10794449],
       [0.03493711],
       [0.5093799 ],
       [0.2200698 ],
       [0.28567347],
       [0.4367848 ],
       [0.1590488 ],
       [0.19696385],
       [0.03284493],
       [0.14317155],
       [0.06698319],
       [0.80578816],
       [0.5293448 ],
       [0.06704721],
       [0.11229691],
       [0.066

In [77]:
y_preds1 = np.where(y_preds1 >= 0.5 , 1 , 0 ) 
y_preds1

array([[0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [1],
       [0],
       [1],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [1],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [1],
       [0],
       [1],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [1],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
    

In [82]:
output = pd.DataFrame({ 'PassengerId' : df_test_original['PassengerId'] , 'Survived': y_preds1 })
output.to_csv('titanic-predictions1.csv', index = False)
output.head()






ValueError: ignored

### 2. Arquitectura 2

In [ ]:
def red_v2():
  #base secuencial para definir la estructura de la Red 
  clasificador = Sequential()

  #primera capa oclta
  clasificador.add(Dense(input_dim = 7, units=6, activation='relu', kernel_initializer='uniform'))

  #segunda capa oculta
  clasificador.add(Dense(units=6, activation='relu', kernel_initializer='uniform'))

  #capa de salida
  clasificador.add(Dense(units=1, activation='sigmoid', kernel_initializer='uniform'))

  #parametros de optimización
  clasificador.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['accuracy'])

  return clasificador

In [ ]:
clasificador = KerasClassifier(build_fn=red_v2, batch_size=25, nb_epoch=1000)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  """Entry point for launching an IPython kernel.


In [ ]:
kfolds = KFold(n_splits=5, shuffle=True)

accs2 = cross_val_score(clasificador, X=X_train, y=y_train, cv=kfolds, verbose=1, n_jobs=-1)
accs2.mean()

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    3.4s finished


0.6092387080192566

### 3. Arquitectura 3

In [ ]:
def red_v3():
  #base secuencial para definir la estructura de la Red 
  clasificador = Sequential()

  #primera capa oclta
  clasificador.add(Dense(input_dim = 7, units=9, activation='relu', kernel_initializer='uniform'))

  #segunda capa oculta
  clasificador.add(Dense(units=9, activation='relu', kernel_initializer='uniform'))

  #capa de salida
  clasificador.add(Dense(units=1, activation='sigmoid', kernel_initializer='uniform'))

  #parametros de optimización
  clasificador.compile(optimizer='RMSprop', loss='binary_crossentropy', metrics=['accuracy'])

  return clasificador

  clasificador = KerasClassifier(build_fn=red_v3, batch_size=25, nb_epoch=2000)
  kfolds = KFold(n_splits=10, shuffle=True)

accs3 = cross_val_score(clasificador, X=X_train, y=y_train, cv=kfolds, verbose=1, n_jobs=-1)
accs3.mean()


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    3.4s finished


0.6252516150474549

### 4. Arquitectura 4

In [ ]:
def red_v4():
  #base secuencial para definir la estructura de la Red 
  clasificador = Sequential()

  #primera capa oclta
  clasificador.add(Dense(input_dim = 7, units=2, activation='relu', kernel_initializer='uniform'))

  #segunda capa oculta
  clasificador.add(Dense(units=2, activation='relu', kernel_initializer='uniform'))

  #capa de salida
  clasificador.add(Dense(units=1, activation='sigmoid', kernel_initializer='uniform'))

  #parametros de optimización
  clasificador.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['accuracy'])

  return clasificador

  clasificador = KerasClassifier(build_fn=red_v4, batch_size=25, nb_epoch=5000)
  kfolds = KFold(n_splits=10, shuffle=True)

accs4 = cross_val_score(clasificador, X=X_train, y=y_train, cv=kfolds, verbose=1, n_jobs=-1)
accs4.mean()

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    3.8s finished


0.6494967818260193

### 5. Arquitectura 5

In [ ]:
def red_v5():
  #base secuencial para definir la estructura de la Red 
  clasificador = Sequential()

  #primera capa oclta
  clasificador.add(Dense(input_dim = 7, units=2, activation='relu', kernel_initializer='uniform'))

  #segunda capa oculta
  clasificador.add(Dense(units=4, activation='relu', kernel_initializer='uniform'))

  #capa de salida
  clasificador.add(Dense(units=1, activation='sigmoid', kernel_initializer='uniform'))

  #parametros de optimización
  clasificador.compile(optimizer='SGD', loss='binary_crossentropy', metrics=['accuracy'])

  return clasificador

  clasificador = KerasClassifier(build_fn=red_v5, batch_size=25, nb_epoch=15000)
  kfolds = KFold(n_splits=10, shuffle=True)

accs5 = cross_val_score(clasificador, X=X_train, y=y_train, cv=kfolds, verbose=1, n_jobs=-1)
accs5.mean()

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    3.4s finished


0.620735478401184